In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import argparse
import os
import sys
from socket import gethostname
import time
import datetime

<IPython.core.display.Javascript object>

In [3]:
parser = argparse.ArgumentParser(
    description="Launches clustering model on "
    "using cortical thickness maps derived from MRI"
)

<IPython.core.display.Javascript object>

In [4]:
args = {
    "runIndex": 1,
    "nrProc": 1,
    "models": 13,
    "nrOuterIt": 5,
    "nrInnerIt": 1,
    "nrClust": 12,
    "initClustering": "k-means",
    "rangeFactor": 1,
    "informPrior": 0,
    "leaderboard": 1,
    "agg": 1,
    "cluster": False,
    "reduceSpace": 1,
    "alphaMRF": 1,
    "d3": 0,
    "stdBeta": 0.1,
    "stdGammaAlpha": 0.0025,
}

<IPython.core.display.Javascript object>

In [5]:
# don't import matplotlib until here, add other imports below
if args.get("agg"):
    # print(matplotlib.__version__)
    import matplotlib

    # print(matplotlib.get_backend())
    matplotlib.use("Agg")
    # print(matplotlib.get_backend())
    # print(asds)

<IPython.core.display.Javascript object>

In [6]:
from voxCommon import initCommonVoxParams
import evaluationFramework
from voxelDPM import *
from aux import *
from adniCommon import *
from env import *
import pandas as pd
import PlotterVDPM
import VDPMNan

<IPython.core.display.Javascript object>

In [7]:
params, plotTrajParams = initCommonVoxParams(args)
plotTrajParams["legendCols"] = 4
plotTrajParams["diagColors"] = {CTL: "b", MCI: "g", AD: "r", -1: "y"}
plotTrajParams["diagLabels"] = {CTL: "CTL", MCI: "MCI", AD: "AD", -1: "N/A"}
plotTrajParams["ylimitsRandPoints"] = (-5, 5)
plotTrajParams["diagNrs"] = [CTL, MCI, AD]

plotTrajParams["SubfigClustMaxWinSize"] = (
    1300,
    plotTrajParams["SubfigClustMaxWinSize"][1],
)
plotTrajParams["Clust3DMaxWinSize"] = (900, 600)
# plotTrajParams['ylimTrajWeightedDataMean'] = (-3,2)
plotTrajParams["ylimTrajSamplesInOneNoData"] = (-2.5, 1.5)
plotTrajParams["biomkAxisLabel"] = "Cortical Thickness Z-score"
plotTrajParams["biomkWasInversed"] = False

refDate = datetime.date(2000, 1, 1)

<IPython.core.display.Javascript object>

In [8]:
def cleanTadpoleData(df):

    df.loc[df.RAVLT_learning < 0, "RAVLT_learning"] = np.nan
    df.loc[df.RAVLT_forgetting < 0, "RAVLT_forgetting"] = np.nan
    df.loc[df.RAVLT_perc_forgetting < 0, "RAVLT_perc_forgetting"] = np.nan

    petCols = list(
        df.loc[:, "HIPPL01_BAIPETNMRC_09_12_16":"MCSUVRCERE_BAIPETNMRC_09_12_16"]
    )
    # df[petCols].replace({'-4': np.nan, -4: np.nan}, inplace=True)
    for c in petCols:
        df.loc[df[c] == "-4", c] = np.nan

    return df

<IPython.core.display.Javascript object>

In [9]:
def dateDiffToMonths(diff):
    return diff.days / (365.0 / 12)

<IPython.core.display.Javascript object>

In [10]:
def parseTadpoleData(df):

    cols = (
        list(df.loc[:, "FDG":"EcogSPTotal"])
        + list(df.loc[:, "Ventricles":"MidTemp"])
        + list(
            df.loc[
                :,
                "ST101SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16":"ST9SV_UCSFFSL_02_01_16_UCSFFSL51ALL_08_01_16",
            ]
        )
        + list(
            df.loc[
                :,
                "ST101SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16":"ST9SV_UCSFFSX_11_02_15_UCSFFSX51_08_01_16",
            ]
        )
        + list(
            df.loc[:, "HIPPL01_BAIPETNMRC_09_12_16":"MCSUVRCERE_BAIPETNMRC_09_12_16"]
        )
        + list(
            df.loc[
                :,
                "CEREBELLUMGREYMATTER_UCBERKELEYAV45_10_17_16":"WM_HYPOINTENSITIES_SIZE_UCBERKELEYAV45_10_17_16",
            ]
        )
        + list(
            df.loc[
                :,
                "CEREBELLUMGREYMATTER_UCBERKELEYAV1451_10_17_16":"WM_HYPOINTENSITIES_SIZE_UCBERKELEYAV1451_10_17_16",
            ]
        )
        + list(df.loc[:, "FA_CST_L_DTIROI_04_30_14":"AD_SUMFX_DTIROI_04_30_14"])
        + list(df.loc[:, "ABETA_UPENNBIOMK9_04_19_17":"PTAU_UPENNBIOMK9_04_19_17"])
    )

    # TODO: re-process data more, continue form here: change AV45 -> AV45/SIZE of ROI
    # print('cols', cols)
    # filter out the FS cols with Standard deviation of volumes, cort thickness, etc ... Only keep average
    colsFilt = []
    for col in cols:
        if col[:2] == "ST" and (col[5] == "S" or col[6] == "S"):
            continue

        colsFilt += [col]

    # print(ads)
    # print(df.D1)
    # print(df.shape)
    d2Ind = df.RID[df.loc[:, "D2"] == 1].to_numpy()

    # print('d2Ind', np.unique(d2Ind), np.unique(d2Ind).shape)

    df[cols] = df[cols].apply(pd.to_numeric, errors="coerce", axis=1)
    pickle.dump(
        dict(df=df),
        open("data/tadpoleCleanDf.npz", "wb"),
        protocol=pickle.HIGHEST_PROTOCOL,
    )
    df = pickle.load(open("data/tadpoleCleanDf.npz", "rb"))["df"]

    # normalise ventricles by ICV
    df["Ventricles"] = df["Ventricles"] / df["ICV"]

    # data = df.as_matrix(columns=cols)
    data = df[cols].to_numpy()

    # convert diagnoses such as 'MCI to Dementia' to 'Dementia', etc ...
    # ctlDxchange = [1, 7, 9] mciDxchange = [2, 4, 8] adDxChange = [3, 5, 6]
    mapping = {1: CTL, 7: CTL, 9: CTL, 2: MCI, 4: MCI, 8: MCI, 3: AD, 5: AD, 6: AD}
    # df.replace({'DXCHANGE': mapping}, inplace=True)
    df["DXCHANGE"] = df["DXCHANGE"].map(mapping)
    diag = df["DXCHANGE"].to_numpy()

    examDates = df.EXAMDATE.to_numpy()
    df["EXAMDATE"] = pd.to_datetime(df["EXAMDATE"], format="%Y-%m-%d")

    dataDf = df[cols]
    dataDf.to_csv("output/tadpoleCleanDf.csv")

    # build numpy string array
    nrCols = len(cols)
    labels = np.ndarray((nrCols,), dtype="S100")
    for c in range(nrCols):
        labels[c] = cols[c]

    partCode = df.RID.to_numpy()
    # print('partCode', partCode)
    unqPartCode = np.unique(partCode)
    nrUnqSubj = len(unqPartCode)

    ageAtScan = np.zeros(partCode.shape, np.float)
    scanTimepts = np.zeros(partCode.shape, np.float)

    for s in range(nrUnqSubj):
        subjRowsCurr = df.RID == unqPartCode[s]
        ageAtBlCurr = df.AGE[subjRowsCurr]

        examDatesCurr = df.EXAMDATE[subjRowsCurr]
        minInd = np.argmin(examDatesCurr)
        yearsDiffs = [(d - examDatesCurr[minInd]).days / 365 for d in examDatesCurr]

        ageAtScan[subjRowsCurr] = ageAtBlCurr + yearsDiffs

        scanTimepts[subjRowsCurr] = np.argsort(np.argsort(yearsDiffs))

        sortedVisitsCurr = np.argsort(yearsDiffs)
        diagCurrSorted = diag[subjRowsCurr][sortedVisitsCurr]

        notNanDiags = [d for d in diagCurrSorted if not np.isnan(d)]

        diagCurrSortedFilled = np.copy(diagCurrSorted)

        if len(notNanDiags) == 0:
            # set the subject diag as -1 if there is absolutely no diagnosis
            diagCurrSortedFilled[0] = -1
        else:
            if np.isnan(diagCurrSortedFilled[0]):
                diagCurrSortedFilled[0] = notNanDiags[0]

            for v in range(1, len(sortedVisitsCurr)):
                if np.isnan(diagCurrSortedFilled[v]):
                    diagCurrSortedFilled[v] = diagCurrSortedFilled[v - 1]

        diagFilledInOrigOrder = diagCurrSortedFilled[np.argsort(sortedVisitsCurr)]

        diag[subjRowsCurr] = diagFilledInOrigOrder

    # compute number of months since Jan 2000 for each EXAMDATEs
    monthsSinceRefTime = np.zeros(partCode.shape, np.float)

    for r in range(df.RID.shape[0]):
        monthsSinceRefTime[r] = dateDiffToMonths(df.EXAMDATE[r].date() - refDate)

    assert not np.isnan(ageAtScan).any()
    assert not np.isnan(diag).any()
    assert not np.isnan(scanTimepts).any()
    assert not np.isnan(partCode).any()
    assert not np.isnan(monthsSinceRefTime).any()

    return (
        data,
        diag,
        labels,
        scanTimepts,
        partCode,
        ageAtScan,
        dataDf,
        monthsSinceRefTime,
        examDates,
        d2Ind,
    )

<IPython.core.display.Javascript object>

In [11]:
def makeBiomksDecr(data, diag, labels):

    assert data.shape[0] == diag.shape[0]

    # perform t-test on every voxel, sort them by p-values
    pVals = scipy.stats.ttest_ind(
        data[diag == CTL, :], data[diag == AD, :], nan_policy="omit"
    )[1]

    sortedInd = np.argsort(pVals)
    # print('sortedInd', sortedInd)

    # print('data[diag == CTL, :]', data[diag == CTL, :])
    meanCTL = np.nanmean(data[diag == CTL, :], axis=0)
    meanAD = np.nanmean(data[diag == AD, :], axis=0)
    stdCTL = np.nanstd(data[diag == CTL, :], axis=0)
    stdAD = np.nanstd(data[diag == AD, :], axis=0)

    # record which biomarkers have had their sign flipped. Multiply this vector
    # with the scale from the normalisation with controls that we did earlier.
    biomkScaleExtra = np.ones(pVals.shape)

    for b in sortedInd:

        if (pVals[b] < 0.001) and meanAD[b] > meanCTL[b]:
            data[:, b] = data[:, b] * (-1)
            biomkScaleExtra[b] = -1
    # print('flipped sign for %s' % labels[b])

    return data, sortedInd, biomkScaleExtra, pVals

<IPython.core.display.Javascript object>

In [12]:
def launchTadpole(runIndex, nrProcesses, modelToRun):

    genProcessedDataset = 1

    if genProcessedDataset:
        if args.get("leaderboard") == 0:
            inputFileData = "data/TADPOLE_D1_D2.csv"
            sys.stdout.flush()
            outFileCheckpoint2 = "output/tadpoleDf2.npz"
            # print('loading data file')
            df = pd.read_csv(inputFileData, low_memory=False)
            df = cleanTadpoleData(df)
            data, diag, labels, scanTimepts, partCode, ageAtScan, dataDf, monthsSinceRefTime, examDates, predInd = parseTadpoleData(
                df
            )

        else:
            outFileCheckpoint2 = "output/tadpoleDf2Ldb.npz"
            # print('loading data file')
            inputFileDataD1D2 = "data/TADPOLE_D1_D2.csv"
            df = pd.read_csv(inputFileDataD1D2, low_memory=False)
            df = cleanTadpoleData(df)
            inputFileDataLB = "data/TADPOLE_LB1_LB2.csv"
            dfLB = pd.read_csv(inputFileDataLB, low_memory=False)

            # this function runs exactly as in the normal submission, no difference here for leaderboard
            data, diag, labels, scanTimepts, partCode, ageAtScan, dataDf, monthsSinceRefTime, examDates, _ = parseTadpoleData(
                df
            )

            filterMaskLB12 = np.logical_or(dfLB.LB1 == 1, dfLB.LB2 == 1)
            assert data.shape[0] == dfLB.shape[0]

            # print(np.sum(filterMaskLB12), filterMaskLB12.shape[0])
            # print(dads)

            data = data[filterMaskLB12, :]
            diag = diag[filterMaskLB12]
            scanTimepts = scanTimepts[filterMaskLB12]
            partCode = partCode[filterMaskLB12]
            ageAtScan = ageAtScan[filterMaskLB12]
            dataDf = dataDf[filterMaskLB12]
            dataDf.reset_index(drop=True, inplace=True)
            dataDf.reindex(index=range(dataDf.shape[0]))
            monthsSinceRefTime = monthsSinceRefTime[filterMaskLB12]
            examDates = examDates[filterMaskLB12]
            predInd = dfLB.RID[dfLB.LB2 == 1].to_numpy()

        dataStruct = dict(
            data=data,
            diag=diag,
            labels=labels,
            scanTimepts=scanTimepts,
            partCode=partCode,
            ageAtScan=ageAtScan,
            dataDf=dataDf,
            monthsSinceRefTime=monthsSinceRefTime,
            examDates=examDates,
            predInd=predInd,
        )
        pickle.dump(
            dataStruct, open(outFileCheckpoint2, "wb"), protocol=pickle.HIGHEST_PROTOCOL
        )

    else:
        if args.get("leaderboard") == 0:
            outFileCheckpoint2 = "output/tadpoleDf2.npz"
        else:
            outFileCheckpoint2 = "output/tadpoleDf2Ldb.npz"

    dataStruct = pickle.load(open(outFileCheckpoint2, "rb"))
    data = dataStruct["data"]
    diag = dataStruct["diag"]
    labels = dataStruct["labels"]
    scanTimepts = dataStruct["scanTimepts"]
    partCode = dataStruct["partCode"]
    ageAtScan = dataStruct["ageAtScan"]
    # dataDf = dataStruct['dataDf']
    monthsSinceRefTime = dataStruct["monthsSinceRefTime"]
    examDates = dataStruct["examDates"]
    predInd = dataStruct["predInd"]

    # filter AD subjects
    # diagInd = np.array(np.where(matData['diag'] == PCA)[0])
    ##print('compiling parameters')
    sys.stdout.flush()

    ##print('diag', np.unique(diag), diag)
    # print(adsas)

    unqPartCode = np.unique(partCode)
    nrUnqPart = len(unqPartCode)

    # calculate Z-scores at each point w.r.t controls at baseline
    # controlBlInd = np.logical_and(diag == CTL, scanTimepts == 1)
    controlInd = diag == CTL
    stdBiomk = np.nanstd(data[diag == CTL], 0)
    biomkMaskCTL = np.isnan(np.nanstd(data[diag == CTL], 0))
    biomkMaskAD = np.isnan(np.nanstd(data[diag == AD], 0))
    biomkMaskMCI = np.isnan(np.nanstd(data[diag == MCI], 0))
    mask = np.logical_or(np.logical_or(biomkMaskCTL, biomkMaskMCI), biomkMaskAD)
    # print(ads)
    selectedBiomk = np.logical_not(np.logical_or(mask, stdBiomk == 0))

    ##print(data.shape)
    data = data[:, selectedBiomk]
    labels = labels[selectedBiomk]
    pointIndices = np.array(range(data.shape[1]))
    stdBiomk = np.nanstd(data[controlInd], 0)
    ##print(data.shape)
    # print(ads)

    meanCTL = np.nanmean(data[controlInd], 0)  # calculate Z-scores
    stdCTL = np.nanstd(data[controlInd], 0)
    dataZ = (data - meanCTL[None, :]) / stdCTL[None, :]
    data = dataZ

    outlierRows, outlierCols = np.where(np.abs(dataZ) > 50)
    filterMask = np.ones(data.shape[0], bool)
    filterMask[outlierRows] = 0
    data = data[filterMask]
    diag = diag[filterMask]
    scanTimepts = scanTimepts[filterMask]
    partCode = partCode[filterMask]
    ageAtScan = ageAtScan[filterMask]
    monthsSinceRefTime = monthsSinceRefTime[filterMask]
    examDates = examDates[filterMask]
    meanAgeAtScan = np.mean(ageAtScan.astype(float))
    ageAtScanCentered = (ageAtScan - meanAgeAtScan).astype(np.float16)

    nrSubj, nrBiomk = data.shape
    # print('nrBiomk', nrBiomk)
    # print(adsa)

    dataAD = data[diag == AD, :]

    # make all biomarkers decreasing by flipping their signs if necessary
    # also perform a t-test to see which ones are most informative, sort them by pvalue (i.e. sortedByPvalInd)
    # the new data is re-scaled
    data, sortedByPvalInd, biomkScaleExtra, pVals = makeBiomksDecr(data, diag, labels)
    # doTtest(data, diag, pointIndices)

    # multiply the scaling we did from controls with (-1) if the biomk had the sign flipped
    stdBiomkRescale = biomkScaleExtra * stdCTL

    assert sortedByPvalInd.shape[0] == data.shape[1]

    sys.stdout.flush()

    global params

    params["data"] = data
    params["diag"] = diag
    params["scanTimepts"] = scanTimepts
    params["partCode"] = partCode
    params["ageAtScan"] = ageAtScan
    params[
        "initShift"
    ] = ageAtScanCentered  # initialise time shifts (betas) to (age - meanAge)
    params["biomkDir"] = DECR
    params["modelToRun"] = modelToRun
    params["datasetFull"] = "tadpole"
    params["labels"] = labels
    params["predInd"] = predInd
    params["examDates"] = examDates

    # print('ageAtScanCentered', ageAtScanCentered)
    # print('ageAtScan', ageAtScan)
    # print('scanTimepts', scanTimepts)
    # ada

    ## print('outFileCheckpoint2', outFileCheckpoint2)
    ## print('d2Ind', np.unique(predInd), np.unique(predInd).shape)
    # print(adsa)

    # filter down to 100 subjects to make it run faster, just for testing. Also select only some biomarkers
    unqPartCode = np.unique(params["partCode"])
    nrPartToSample = 30
    np.random.seed(3)
    selectedPartCode = np.random.choice(unqPartCode, nrPartToSample)
    dataIndices = np.in1d(params["partCode"], selectedPartCode)
    # params = filterDDSPAIndices(params, dataIndices)

    indices = [
        i
        for i in range(len(labels))
        if labels[i]
        in [
            b"FDG",
            b"AV45",
            b"CDRSB",
            b"ADAS13",
            b"Ventricles",
            b"Hippocampus",
            b"WholeBrain",
            b"Entorhinal",
            b"MidTemp",
            b"ABETA_UPENNBIOMK9_04_19_17",
            b"TAU_UPENNBIOMK9_04_19_17",
            b"PTAU_UPENNBIOMK9_04_19_17",
        ]
    ]

    # indices = sortedByPvalInd[:300]
    # print('pVals lowest', pVals[sortedByPvalInd[:300]])
    # print('pVals highest', pVals[sortedByPvalInd[-100:]])
    # print('indices', indices)
    # print(ads)
    ##print('labels', labels[indices])
    # print(adsa)
    # print(np.nanstd(data,axis=0)[indices])
    data = params["data"][:, indices]
    params["data"] = data
    labels = labels[indices]
    params["labels"] = labels
    nrBiomk = params["data"].shape[1]
    ##print('data.shape', params['data'].shape)
    meanCTL = meanCTL[indices]
    stdBiomkRescale = stdBiomkRescale[indices]
    ##print(stdBiomkRescale)
    ## print('flippedBiomk', labels[stdBiomkRescale < 0])
    sortedByPvalInd = np.argsort(np.argsort(sortedByPvalInd[indices]))

    # visTadpoleHist(data, diag, ageAtScan, labels, plotTrajParams, sortedByPvalInd)
    # print(adsa)

    # visTadpoleSpagetti(data, diag, ageAtScan, scanTimepts, partCode, labels, plotTrajParams, sortedByPvalInd)
    # print(adsa)

    # print('CTL %f +/- %f', np.nanmean(params['data'][params['diag'] == CTL, 1]), np.nanstd(params['data'][params['diag'] == CTL, 1]))
    # print('AD %f +/- %f', np.nanmean(params['data'][params['diag'] == AD, 1]), np.nanstd(params['data'][params['diag'] == AD, 1]))
    # print(ads)

    plotTrajParams["nearestNeighbours"] = np.array(range(nrBiomk))
    params["adjList"] = np.nan
    params["nearNeighInitClust"] = np.array(range(nrBiomk))
    params["initClustSubsetInd"] = np.array(range(nrBiomk))
    params["meanBiomkRescale"] = meanCTL  # for rescaling back if necessary
    params["stdBiomkRescale"] = stdBiomkRescale
    # params['fixSpeed'] = True # if true then don't model progression speed, only time shift
    params[
        "fixSpeed"
    ] = False  # if true then don't model progression speed, only time shift

    diagNrs = np.unique(diag)
    # print('diagNrs, diag', diagNrs, diag)
    # print(asdas)

    # print(len(params['acqDate']), data.shape[0])
    sys.stdout.flush()
    assert (
        params["data"].shape[0]
        == params["diag"].shape[0]
        == params["scanTimepts"].shape[0]
        == params["partCode"].shape[0]
        == params["ageAtScan"].shape[0]
    )

    # sets an uninformative or informative prior
    priorNr = setPrior(
        params,
        args.get("informPrior"),
        mean_gamma_alpha=1,
        std_gamma_alpha=0.1,
        mu_beta=0,
        std_beta=5,
    )

    suffix = ""
    if args.get("leaderboard"):
        suffix = "Ldb"
        # print(ads)

    expName = "tadpoleInit%sCl%dPr%dRa%d%s" % (
        args.get("initClustering"),
        params["nrClust"],
        priorNr,
        args.get("rangeFactor"),
        suffix,
    )
    plotTrajParams["sortedByPvalInd"] = sortedByPvalInd
    plotTrajParams["pointIndices"] = pointIndices
    plotTrajParams["expName"] = expName
    plotTrajParams["ageTransform"] = (0, 1)  # no age normalisation was necessary
    plotTrajParams["datasetFull"] = params["datasetFull"]
    plotTrajParams["labels"] = labels

    params["plotTrajParams"] = plotTrajParams

    # R - run that checkpoint, L - load result from checkpoint
    # [initClust, modelFit, AIC/BIC, blender, theta_sampling]
    params["runPartStd"] = ["R", "R", "I", "I", "I"]
    params["runPartMain"] = [
        "R",
        "I",
        "I",
        "I",
    ]  # [mainPart, plot, stage, globalMinStats]
    params["runPartCogCorr"] = ["I"]
    params["runPartCogCorrMain"] = ["L", "L", "I", "I", "L"]
    params["runPartDirDiag"] = ["R", "R", "I"]
    params["runPartStaging"] = ["L", "L", "I"]
    params["runPartDiffDiag"] = ["R", "R", "I"]
    params["runPartConvPred"] = ["I", "I", "I"]
    params["runPartCVNonOverlap"] = ["R"]
    params["runPartCVNonOverlapMain"] = ["L", "L", "I", "I", "L"]
    params["masterProcess"] = runIndex == 0

    if params["masterProcess"]:
        # [initClust, modelFit, AIC/BIC, blender, theta_sampling]
        params["runPartStd"] = ["L", "L", "I", "I", "I"]
        params["runPartMain"] = [
            "R",
            "R",
            "R",
            "I",
        ]  # [mainPart, plot, stage, globalMinStats]
        params["runPartCogCorr"] = ["I"]
        params["runPartCogCorrMain"] = ["L", "L", "I", "I", "I"]
        params["runPartDirDiag"] = ["R", "R", "I"]
        params["runPartStaging"] = ["L", "L", "I"]
        params["runPartDiffDiag"] = ["R", "R", "I"]
        params["runPartConvPred"] = ["I", "I", "I"]
        params["runPartCVNonOverlap"] = ["I"]
        params["runPartCVNonOverlapMain"] = ["R", "R", "I", "R", "R"]

    runAllExpFunc = runAllExpTADPOLE
    modelNames, res = evaluationFramework.runModels(
        params, expName, modelToRun, runAllExpFunc
    )

    # now generate forecast
    # print('Generating forecast ... ')
    teamName = "DIVE6"
    if args.get("leaderboard"):
        outputFile = "output/TADPOLE_Submission_Leaderboard_%s.csv" % teamName
        predStartDate = datetime.date(2010, 5, 1)
        nrYearsToPred = 7
        nrMonthsToPred = 12 * nrYearsToPred  # 5 years
    else:
        outputFile = "output/TADPOLE_Submission_%s.csv" % teamName
        predStartDate = datetime.date(2018, 1, 1)
        nrYearsToPred = 5
        nrMonthsToPred = 12 * nrYearsToPred  # 7 years

    resCurrModel = res[0]["std"]

    predAdasAllSubj, predVentsAllSubj, predDiagAllSubj = makeTadpoleForecast(
        predStartDate, nrYearsToPred, nrMonthsToPred, resCurrModel, params
    )

    # write forecast to file
    writeTadpoleSubmission(
        predAdasAllSubj,
        predVentsAllSubj,
        predDiagAllSubj,
        outputFile,
        nrMonthsToPred,
        predStartDate,
        params,
    )

<IPython.core.display.Javascript object>

In [13]:
def makeTadpoleForecast(
    predStartDate, nrYearsToPred, nrMonthsToPred, resCurrModel, params
):

    yearsFromPredStartToEachPredDate = np.linspace(
        0, nrYearsToPred, num=nrMonthsToPred, endpoint=False
    )

    nrClust = params["nrClust"]
    assert abs(yearsFromPredStartToEachPredDate[1] - (1.0 / 12)) < 0.00001
    # make predictions
    startMonth = dateDiffToMonths(refDate - predStartDate)

    trajFunc = sigmoidFunc

    unqPartCodeFromRes = resCurrModel["uniquePartCode"]
    predInd = params["predInd"]
    predSetRidUnq = np.unique(predInd)

    nrSubjPredSet = predSetRidUnq.shape[0]
    # for each patient
    clustProbBC = resCurrModel["clustProb"]
    thetas = resCurrModel["thetas"]
    variances = resCurrModel["variances"]

    labels = params["labels"]
    indexAdas = np.where(labels == b"ADAS13")[0][0]
    indexVents = np.where(labels == b"Ventricles")[0][0]

    predDiagAllSubj = np.zeros((nrSubjPredSet, nrMonthsToPred, 3), np.float)
    predAdasAllSubj = np.zeros((nrSubjPredSet, nrMonthsToPred, 3), np.float)
    predVentsAllSubj = np.zeros((nrSubjPredSet, nrMonthsToPred, 3), np.float)

    dpsCross = resCurrModel["dpsCross"]
    crossDiag = resCurrModel["crossDiag"]

    dpsCTL = dpsCross[crossDiag == CTL]
    dpsMCI = dpsCross[crossDiag == MCI]
    dpsAD = dpsCross[crossDiag == AD]

    partCode = params["partCode"]
    partCodeCurr = resCurrModel["crossPartCode"]
    # ageAtScan = resCurrModel['ageAtScan']

    data = params["data"]

    assert partCodeCurr.shape[0] == partCode.shape[0]

    kernelWidth = np.std(dpsCross) / 6  # need to test this parameter by visualisation

    from sklearn.neighbors.kde import KernelDensity

    kdeCTL = KernelDensity(kernel="gaussian", bandwidth=kernelWidth).fit(
        dpsCTL.reshape(-1, 1)
    )
    kdeMCI = KernelDensity(kernel="gaussian", bandwidth=kernelWidth).fit(
        dpsMCI.reshape(-1, 1)
    )
    kdeAD = KernelDensity(kernel="gaussian", bandwidth=kernelWidth).fit(
        dpsAD.reshape(-1, 1)
    )

    kdeXs = np.linspace(np.min(dpsCross), np.max(dpsCross), num=100).reshape(-1, 1)

    fig = pl.figure(3)
    pl.clf()
    # print('kdeCTL.score_samples(kdeXs)', np.exp(kdeCTL.score_samples(kdeXs)))
    pl.plot(
        kdeXs,
        np.exp(kdeCTL.score_samples(kdeXs)),
        label="CTL",
        c=plotTrajParams["diagColors"][CTL],
    )
    pl.plot(
        kdeXs,
        np.exp(kdeMCI.score_samples(kdeXs)),
        label="MCI",
        c=plotTrajParams["diagColors"][MCI],
    )
    pl.plot(
        kdeXs,
        np.exp(kdeAD.score_samples(kdeXs)),
        label="AD",
        c=plotTrajParams["diagColors"][AD],
    )
    pl.legend()
    fig.show()
    fig.savefig("%s/diagHist.png" % (resCurrModel["outFolder"]), dpi=100)

    ageAtScan = params["ageAtScan"]
    examDates = params["examDates"]

    runPred = "R"
    doPlot = 0
    predFile = "output/tadpolePredD2.npz"

    meanBiomkRescale = params["meanBiomkRescale"]
    stdBiomkRescale = params["stdBiomkRescale"]

    if runPred == "R":
        for s in range(nrSubjPredSet):

            ######### find dps at forecasted months ##########

            # find age at forecasted months
            subjRowsCurr = partCode == predSetRidUnq[s]

            # import pdb
            # pdb.set_trace()

            # for one timepoint, find the age and the examDate
            # print('part : ', predSetRidUnq[s], np.sum(subjRowsCurr))
            # print('part ageAtScan: ', predSetRidUnq[s], ageAtScan[subjRowsCurr][0])

            # compute age of subject at every prediction date
            ageOneTimept = ageAtScan[subjRowsCurr][0]
            examDateOneTimept = datetime.datetime.strptime(
                examDates[subjRowsCurr][0], "%Y-%m-%d"
            ).date()
            yearsToPredStartDate = (predStartDate - examDateOneTimept).days / 365
            ageAtPredDates = (
                ageOneTimept + yearsToPredStartDate + yearsFromPredStartToEachPredDate
            )

            # compute dps
            subShiftsCurr = resCurrModel["subShifts"][
                unqPartCodeFromRes == predSetRidUnq[s]
            ]
            dpsAtFutForecastDatesCurr = calcDpsGivenAges(ageAtPredDates, subShiftsCurr)

            ######## find model predictions for those DPSs ##############3

            futureForecastsAdas, futureForecastsVents = calcModelPredAdasVents(
                dpsAtFutForecastDatesCurr,
                thetas,
                variances,
                clustProbBC[indexAdas, :].T,
                clustProbBC[indexVents, :].T,
                trajFunc,
            )

            # add subject-specific intercept to the predictions, is subject has data
            # warning: can contain NaNs and even be NaN in all entries.
            adasDataCurrSubj = data[subjRowsCurr, indexAdas]
            ventsDataCurrSubj = data[subjRowsCurr, indexVents]

            ageCurrVisits = ageAtScan[subjRowsCurr]
            dpsSubjCurrVisits = calcDpsGivenAges(ageCurrVisits, subShiftsCurr)
            currVisitsPredAdas, currVisitsPredVents = calcModelPredAdasVents(
                dpsSubjCurrVisits,
                thetas,
                variances,
                clustProbBC[indexAdas, :].T,
                clustProbBC[indexVents, :].T,
                trajFunc,
            )

            futureForecastsAdas = addSubjIntercept(
                dpsAtFutForecastDatesCurr,
                futureForecastsAdas,
                adasDataCurrSubj,
                currVisitsPredAdas,
            )
            futureForecastsVents = addSubjIntercept(
                dpsAtFutForecastDatesCurr,
                futureForecastsVents,
                ventsDataCurrSubj,
                currVisitsPredVents,
            )

            # convert predictions back to un-normalised values

            predAdasNotNorm = (
                futureForecastsAdas * stdBiomkRescale[indexAdas]
                + meanBiomkRescale[indexAdas]
            )
            predVentsNotNorm = (
                futureForecastsVents * stdBiomkRescale[indexVents]
                + meanBiomkRescale[indexVents]
            )

            predAdasAllSubj[s, :, :] = predAdasNotNorm
            predAdasAllSubj[s, :, 1] = predAdasNotNorm[
                :, 2
            ]  # need invert lower& upper bounds due to sign change
            predAdasAllSubj[s, :, 2] = predAdasNotNorm[:, 1]

            predVentsAllSubj[s, :, :] = predVentsNotNorm
            predVentsAllSubj[s, :, 1] = predVentsNotNorm[:, 2]
            predVentsAllSubj[s, :, 2] = predVentsNotNorm[:, 1]

            # print('predAdasNotNorm', predAdasNotNorm[0,:])
            # print(adsa)

            adasDataCurrSubjUnnorm = (
                adasDataCurrSubj * stdBiomkRescale[indexAdas]
                + meanBiomkRescale[indexAdas]
            )
            ventsDataCurrSubjUnnorm = (
                ventsDataCurrSubj * stdBiomkRescale[indexVents]
                + meanBiomkRescale[indexVents]
            )

            ctlLik = np.exp(
                kdeCTL.score_samples(dpsAtFutForecastDatesCurr.reshape(-1, 1))
            )
            mciLik = np.exp(
                kdeMCI.score_samples(dpsAtFutForecastDatesCurr.reshape(-1, 1))
            )
            adLik = np.exp(
                kdeAD.score_samples(dpsAtFutForecastDatesCurr.reshape(-1, 1))
            )

            sumLik = ctlLik + mciLik + adLik

            predDiagAllSubj[s, :, 0] = ctlLik / sumLik
            predDiagAllSubj[s, :, 1] = mciLik / sumLik
            predDiagAllSubj[s, :, 2] = adLik / sumLik

            if doPlot:
                if args.get("leaderboard"):
                    lb4Data = pd.read_csv("data/TADPOLE_LB4.csv")
                    lb4Data["CognitiveAssessmentDate"] = [
                        datetime.datetime.strptime(x, "%Y-%m-%d")
                        for x in lb4Data["CognitiveAssessmentDate"]
                    ]
                    lb4Data["ScanDate"] = [
                        datetime.datetime.strptime(x, "%Y-%m-%d").date()
                        for x in lb4Data["ScanDate"]
                    ]
                    mapping = {"CN": 0, "MCI": 1, "AD": 2}
                    lb4Data.replace({"Diagnosis": mapping}, inplace=True)

                    currSubjMaskLB4 = lb4Data.RID == predSetRidUnq[s]
                    adasLB4CurrSubj = lb4Data.ADAS13[currSubjMaskLB4]
                    ventsLB4CurrSubj = lb4Data.Ventricles[currSubjMaskLB4]
                    diagLB4CurrSubj = lb4Data.Diagnosis[currSubjMaskLB4]

                    datesLB4CurrSubj = lb4Data["CognitiveAssessmentDate"][
                        currSubjMaskLB4
                    ]

                    yearsFromRefDateToLB4Dates = np.array(
                        [
                            (d.date() - examDateOneTimept).days / 365
                            for d in datesLB4CurrSubj
                        ]
                    )
                    ageAtLB4datesCurrSubj = ageOneTimept + yearsFromRefDateToLB4Dates

                    lb4Params = dict(
                        adasLB4CurrSubj=adasLB4CurrSubj,
                        ventsLB4CurrSubj=ventsLB4CurrSubj,
                        diagLB4CurrSubj=diagLB4CurrSubj,
                        ageAtLB4datesCurrSubj=ageAtLB4datesCurrSubj,
                    )

                else:
                    lb4Params = None

                plotSubjForecasts(
                    predAdasAllSubj[s, :, :],
                    predVentsAllSubj[s, :, :],
                    predDiagAllSubj[s, :, :],
                    ageAtPredDates,
                    adasDataCurrSubjUnnorm,
                    ventsDataCurrSubjUnnorm,
                    ageCurrVisits,
                    lb4Params,
                    rid=predSetRidUnq[s],
                )

        ds = dict(
            predAdasAllSubj=predAdasAllSubj,
            predVentsAllSubj=predVentsAllSubj,
            predDiagAllSubj=predDiagAllSubj,
        )
        pickle.dump(ds, open(predFile, "wb"), protocol=pickle.HIGHEST_PROTOCOL)

    else:
        ds = pickle.load(open(predFile, "rb"))
        predAdasAllSubj = ds["predAdasAllSubj"]
        predVentsAllSubj = ds["predVentsAllSubj"]
        predDiagAllSubj = ds["predDiagAllSubj"]

    return predAdasAllSubj, predVentsAllSubj, predDiagAllSubj

<IPython.core.display.Javascript object>

In [14]:
def plotSubjForecasts(
    predAdasCurrSubj,
    predVentsCurrSubj,
    predDiagCurrSubj,
    ageAtPredDates,
    adasDataCurrSubjUnnorm,
    ventsDataCurrSubjUnnorm,
    ageCurrVisits,
    lb4Params,
    rid,
):

    if lb4Params is not None:
        adasLB4CurrSubj = lb4Params["adasLB4CurrSubj"]
        ventsLB4CurrSubj = lb4Params["ventsLB4CurrSubj"]
        diagLB4CurrSubj = lb4Params["diagLB4CurrSubj"]
        ageAtLB4datesCurrSubj = lb4Params["ageAtLB4datesCurrSubj"]

    pl.figure(3)
    ax = pl.subplot(1, 2, 1)
    ax.set_title("ADAS RID:%d" % rid)
    pl.plot(ageAtPredDates, predAdasCurrSubj)
    pl.scatter(ageCurrVisits, adasDataCurrSubjUnnorm, c="b", s=10)
    if lb4Params is not None:
        pl.scatter(ageAtLB4datesCurrSubj, adasLB4CurrSubj, c="r", s=10)

    ax = pl.subplot(1, 2, 2)
    ax.set_title("Vents RID:%d" % rid)
    pl.plot(ageAtPredDates, predVentsCurrSubj)
    pl.scatter(ageCurrVisits, ventsDataCurrSubjUnnorm, c="b", s=10)
    if lb4Params is not None:
        pl.scatter(ageAtLB4datesCurrSubj, ventsLB4CurrSubj, c="r", s=10)

    pl.show()

<IPython.core.display.Javascript object>

In [15]:
def calcModelPredAdasVents(
    dpsPredCurr, thetas, variances, clustProbAdas, clustProbVents, trajFunc
):

    nrClust = thetas.shape[0]
    predCurrSubClustSC = np.zeros((dpsPredCurr.shape[0], nrClust), float)
    predCurrSubClustSClower = np.zeros((dpsPredCurr.shape[0], nrClust), float)
    predCurrSubClustSCupper = np.zeros((dpsPredCurr.shape[0], nrClust), float)

    for c in range(nrClust):
        predCurrSubClustSC[:, c] = trajFunc(dpsPredCurr, thetas[c, :])
        predCurrSubClustSClower[:, c] = predCurrSubClustSC[:, c] - 0.33 * np.sqrt(
            variances[c]
        )
        predCurrSubClustSCupper[:, c] = predCurrSubClustSC[:, c] + 0.33 * np.sqrt(
            variances[c]
        )

    # from the predictions of each cluster trajectories, predict traj of ADAS and Vents
    # using the probabilities of ADAS/Vents of being assigned to each cluster
    futureForecastsAdas = np.zeros((predCurrSubClustSC.shape[0], 3))
    futureForecastsVents = np.zeros((predCurrSubClustSC.shape[0], 3))

    futureForecastsAdas[:, 0] = np.dot(predCurrSubClustSC, clustProbAdas)
    futureForecastsVents[:, 0] = np.dot(predCurrSubClustSC, clustProbVents)

    futureForecastsAdas[:, 1] = np.dot(predCurrSubClustSClower, clustProbAdas)
    futureForecastsVents[:, 1] = np.dot(predCurrSubClustSClower, clustProbVents)

    futureForecastsAdas[:, 2] = np.dot(predCurrSubClustSCupper, clustProbAdas)
    futureForecastsVents[:, 2] = np.dot(predCurrSubClustSCupper, clustProbVents)

    return futureForecastsAdas, futureForecastsVents

<IPython.core.display.Javascript object>

In [16]:
def calcDpsGivenAges(ageAtPredDates, subShiftsCurr):

    subShiftsPredDates = np.tile(subShiftsCurr, (ageAtPredDates.shape[0], 1))

    # print('subShiftsPredDates', subShiftsPredDates.shape)
    # print('ageAtPredDates', ageAtPredDates.shape)
    assert subShiftsPredDates.shape[0] == ageAtPredDates.shape[0]
    assert subShiftsPredDates.shape[1] == 2

    dpsPredCurr = VoxelDPM.calcDpsNo1array(subShiftsPredDates, ageAtPredDates)

    return dpsPredCurr

<IPython.core.display.Javascript object>

In [17]:
def addSubjIntercept(dpsT, futurePredictions, dataCurrSubjT, modelPredExistingVisits):

    if np.isnan(dataCurrSubjT).all():

        # no data available cur current subject, leave as population estimate
        return futurePredictions
    else:
        # data is
        return futurePredictions + (
            np.nanmean(dataCurrSubjT) - np.mean(modelPredExistingVisits)
        )

<IPython.core.display.Javascript object>

In [18]:
def writeTadpoleSubmission(
    predAdasAllSubj,
    predVentsAllSubj,
    predDiagAllSubj,
    outputFile,
    nrMonthsToPred,
    predStartDate,
    params,
):

    predInd = params["predInd"]
    predSetRidUnq = np.unique(predInd)
    print("Writing forecast to file %s" % outputFile)
    submission_table = pd.DataFrame()
    nrSubjPredSet = predSetRidUnq.shape[0]
    # * Repeated matrices - compare with submission template
    submission_table["RID"] = predSetRidUnq.repeat(nrMonthsToPred)
    submission_table["Forecast Month"] = np.tile(
        range(1, nrMonthsToPred + 1), (nrSubjPredSet, 1)
    ).flatten()

    from dateutil.relativedelta import relativedelta

    endDate = predStartDate + relativedelta(months=+nrMonthsToPred - 1)
    ForecastDates = [predStartDate]
    while ForecastDates[-1] < endDate:
        ForecastDates.append(ForecastDates[-1] + relativedelta(months=+1))

    ForecastDatesStrings = [
        datetime.datetime.strftime(d, "%Y-%m") for d in ForecastDates
    ]
    submission_table["Forecast Date"] = np.tile(
        ForecastDatesStrings, (nrSubjPredSet, 1)
    ).flatten()
    # * Pre-fill forecast data, encoding missing data as NaN
    nanColumn = np.repeat(np.nan, submission_table.shape[0])
    submission_table["CN relative probability"] = nanColumn
    submission_table["MCI relative probability"] = nanColumn
    submission_table["AD relative probability"] = nanColumn
    submission_table["ADAS13"] = nanColumn
    submission_table["ADAS13 50% CI lower"] = nanColumn
    submission_table["ADAS13 50% CI upper"] = nanColumn
    submission_table["Ventricles_ICV"] = nanColumn
    submission_table["Ventricles_ICV 50% CI lower"] = nanColumn
    submission_table["Ventricles_ICV 50% CI upper"] = nanColumn

    # *** Paste in month-by-month forecasts **
    # * 1. Clinical status
    submission_table["CN relative probability"] = predDiagAllSubj[:, :, 0].flatten()
    submission_table["MCI relative probability"] = predDiagAllSubj[:, :, 1].flatten()
    submission_table["AD relative probability"] = predDiagAllSubj[:, :, 2].flatten()
    # * 2. ADAS13 score
    submission_table["ADAS13"] = predAdasAllSubj[:, :, 0].flatten()
    submission_table["ADAS13 50% CI lower"] = predAdasAllSubj[:, :, 1].flatten()
    submission_table["ADAS13 50% CI upper"] = predAdasAllSubj[:, :, 2].flatten()
    # * 3. Ventricles volume (normalised by intracranial volume)
    submission_table["Ventricles_ICV"] = predVentsAllSubj[:, :, 0].flatten()
    submission_table["Ventricles_ICV 50% CI lower"] = predVentsAllSubj[
        :, :, 1
    ].flatten()
    submission_table["Ventricles_ICV 50% CI upper"] = predVentsAllSubj[
        :, :, 2
    ].flatten()

    submission_table.to_csv(outputFile, index=False)

<IPython.core.display.Javascript object>

In [19]:
def runAllExpTADPOLE(params, expName, dpmBuilder):
    """ runs all experiments"""

    res = {}

    params["patientID"] = AD
    params["excludeID"] = -1
    params["excludeXvalidID"] = []
    params["excludeStaging"] = [-1]
    params["anchorID"] = MCI

    # run if this is the master   process or nrProcesses is 1
    unluckyProc = (
        np.mod(params["currModel"] - 1, params["nrProcesses"]) == params["runIndex"] - 1
    )
    unluckyOrNoParallel = (
        unluckyProc or (params["nrProcesses"] == 1) or params["masterProcess"]
    )

    if unluckyOrNoParallel:
        dpmObj, res["std"] = evaluationFramework.runStdDPM(
            params, expName, dpmBuilder, params["runPartMain"]
        )

    return res

<IPython.core.display.Javascript object>

In [ ]:
def printResADNIthick(modelNames, res, plotTrajParams):
    # nrModels = len(modelNames)

    # dinamicModelName = 'VWDPMLinear'
    # staticModelName = 'VWDPMLinearStatic'
    # dinamicModelName = 'VDPM_MRF'
    # staticModelName = 'VWDPMStatic'
    # noDPSModelName = 'VDPMNoDPS'

    print("##### biomk prediction ######")
    nrModels = len(modelNames)
    pred = list(range(nrModels))
    predMean = list(range(nrModels))
    predStd = list(range(nrModels))
    for m in range(nrModels):
        pred[m] = res[m]["cogCorr"]["predStats"]
        predMean[m] = np.nanmean(pred[m])
        predStd[m] = np.nanstd(pred[m])

    for m in range(nrModels):
        print("%s predAllFolds" % modelNames[m], pred[m])
    for m in range(nrModels):
        print("%s predMean" % modelNames[m], predMean[m])
    for m in range(nrModels):
        print("%s predStd" % modelNames[m], predStd[m])

    stats = list(range(nrModels))
    print("##### correlation with cog tests ######")
    for m in range(nrModels):
        stats[m] = res[m]["cogCorr"][
            "statsAllFolds"
        ]  # shape (NR_FOLDS, 2*NR_COG_TESTS)
        # print('stats:', stats[m])
        print(modelNames[m], end="")
        meanStats = np.nanmean(stats[m], 0)
        stdStats = np.nanstd(stats[m], 0)
        for i in range(meanStats.shape[0]):
            print("%.2f +/- %.2f" % (meanStats[i], stdStats[i]), end="")
        print("")

    plotScoresHist(scores=pred, labels=modelNames)

    nrCogStats = stats[0].shape[1]

    # perform paired t-test, as the same cross-validation folds have been used in both cases
    tStats = np.zeros(nrCogStats, float)
    pVals = np.zeros(nrCogStats, float)
    for t in range(nrCogStats):
        tStats[t], pVals[t] = scipy.stats.ttest_rel(stats[0][:, t], stats[0][:, t])

    # expInds = [dinIndex, staIndex, noDPSIndex]
    # printDiffs(expInds, res, modelNames)

    # testPredFPBDin = res[dinIndex]['cogCorr']['testPredPredFPB']
    # testPredFPBSta = res[staIndex]['cogCorr']['testPredPredFPB']
    # testPredFPBNoDPS = res[noDPSIndex]['cogCorr']['testPredPredFPB']
    # testPredDataFPB = res[noDPSIndex]['cogCorr']['testPredDataFPB']
    #
    # outDir = 'resfiles/%s' % plotTrajParams['expName']
    # os.system('mkdir -p %s' % outDir)
    #
    # for i in [0]: #range(len(testPredFPBDin)):
    #   print('testPred diff [%d] ' % i, testPredFPBDin[i] - testPredFPBSta[i])
    #
    #   meanAbsDiffB = np.sum(np.abs(testPredFPBDin[i] - testPredFPBSta[i]), axis=0)
    #   plotter = PlotterVDPM.PlotterVDPM()
    #   plotter.plotDiffs(meanAbsDiffB, plotTrajParams,
    #     filePathNoExt='%s/diffPredDinSta_f%d' % (outDir, i))


# def printDiffs(expInds, res, modelNames):
#
#   for ind in range(len(expInds)):


if __name__ == "__main__":
    # model 4 - VDPM sigmoidal
    # model 5 - VDPM linear

    if args.get("modelToRun"):
        modelToRun = args.get("modelToRun")
    elif args.get("models"):
        modelToRun = np.array(args.get("models"))
    else:
        raise ValueError("need to set either --models or --firstModel & --lastModel")

    launchTadpole(args.get("runIndex"), args.get("nrProc"), modelToRun)

/anaconda3/envs/dive_tadpole/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return getattr(obj, method)(*args, **kwds)
/anaconda3/envs/dive_tadpole/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/anaconda3/envs/dive_tadpole/lib/python3.7/site-packages/ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in greater


partCode [   2    3    3 ... 4557 4512 2380]
filtPartCode [   2    3    3 ... 4557 4512 2380]
longPartCode [   2    3    4 ... 5294 5295 5296]
inverseMap [   0    1    1 ... 1310 1282  935]
np.max(inverseMap) 1736
len(longData) 1737
11984 1736 1737 11984
crossDiag 0 [1. 3. 3. ... 2. 2. 2.]
minMaxRange (12,)
clustProbBCColNorm (12, 12)
crossDiag [1. 3. 3. ... 2. 2. 2.] patientID 3
extraRangeFactor 1.0
outerIt  0
innerIt  0
disprogscore c= 8
disprogscore c= 9
disprogscore c= 10
disprogscore c= 11


/Users/pushpanjali/TADPOLE-SHARE/jupyter/notebooks/DIVE-TADPOLE/PlotterVDPM.py:1793: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  fig.show()
/Users/pushpanjali/TADPOLE-SHARE/jupyter/notebooks/DIVE-TADPOLE/PlotterVDPM.py:1797: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  pl.pause(0.05)


Estimate subject shifts
